In [1]:
import pandas as pd
import numpy as np
import numpy_financial as npf
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
from numpy import *

from ggplot import *
from scipy.stats import logistic
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import *
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
import graphviz
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVC, SVR
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


from linearmodels.iv import IV2SLS
from scipy import stats
from scipy.stats import trim_mean #trimmed mean
from scipy.stats import * #chi2_contingency로 카이제곱통계량계산
from scipy.stats import norm

import keyword
import random

#open api 불러올 때 필요한 것
from urllib.request import urlopen #request package 설치
from bs4 import BeautifulSoup #bs4 package 설치

# 그래프에서 한글 깨지지 않기 위헤 사용함
from matplotlib import rcParams
from matplotlib import font_manager,rc

rcParams['axes.unicode_minus']=False
plt.rcParams['font.family']='Malgun Gothic'
font_name=font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
rc('font',family=font_name)

import FinanceDataReader as fdr
import plotly.graph_objects as go
import plotly.subplots as ms
import math



## lecture 7 주가수익률 분포(2)
### 2 주가등락을 이항분포로 설명
- 주가등락하는 매일은 베르누이 시행
- 주가등락 누적되면 이항분포
- 이항분포의 성공확률 p를 얻기 위해 과거 데이터 사용한다

In [2]:
df=pd.read_csv('c:/temp/samsung_LN7.csv',index_col='Date',parse_dates=True)
df.head()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2010-01-04,16060,16180,16000,16180,239271,0.012516
2010-01-05,16520,16580,16300,16440,559219,0.016069
2010-01-06,16580,16820,16520,16820,459755,0.023114
2010-01-07,16820,16820,16260,16260,443237,-0.033294
2010-01-08,16400,16420,16120,16420,295798,0.009840


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3519 entries, 2010-01-04 to 2024-04-05
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    3519 non-null   int64  
 1   High    3519 non-null   int64  
 2   Low     3519 non-null   int64  
 3   Close   3519 non-null   int64  
 4   Volume  3519 non-null   int64  
 5   Change  3519 non-null   float64
dtypes: float64(1), int64(5)
memory usage: 192.4 KB


In [4]:
df.shape

(3519, 6)

In [5]:
df.tail()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2024-04-01,83200,83300,82000,82000,20116513,-0.004854
2024-04-02,82900,85000,82900,85000,37077944,0.036585
2024-04-03,84300,85000,83500,84100,30493347,-0.010588
2024-04-04,85200,85500,84300,85300,25248934,0.014269
2024-04-05,84500,85000,83800,84500,18773002,-0.009379


In [6]:
# 최근 1년치만 가지고 p를 계산해보자
# 2021-01-01 : 2021-12-31
# loc로 가져온다
# loc: label을 이용해서 가져오는 방식

In [7]:
df1=df.loc['2021-01-01':'2021-12-31']
df1.head()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2021-01-04,81000,84400,80200,83000,38655276,0.024691
2021-01-05,81600,83900,81600,83900,35335669,0.010843
2021-01-06,83300,84500,82100,82200,42089013,-0.020262
2021-01-07,82800,84200,82700,82900,32644642,0.008516
2021-01-08,83300,90000,83000,88800,59013307,0.071170


In [8]:
df1.tail()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2021-12-24,80200,80800,80200,80500,12086380,0.007509
2021-12-27,80600,80600,79800,80200,10783368,-0.003727
2021-12-28,80200,80400,79700,80300,18226325,0.001247
2021-12-29,80200,80200,78500,78800,19794795,-0.018680
2021-12-30,78900,79500,78100,78300,14236700,-0.006345


In [9]:
# return (r): 수익률
# 양수면 주가 상승, 음수면 주가 하락
# python으로 수익률 r 계산가능

In [10]:
df1['ret']=100*(df1['Close'].pct_change())
df1.head()

,Open,High,Low,Close,Volume,Change,ret
Date,,,,,,,
2021-01-04,81000,84400,80200,83000,38655276,0.024691,NaN
2021-01-05,81600,83900,81600,83900,35335669,0.010843,1.084337
2021-01-06,83300,84500,82100,82200,42089013,-0.020262,-2.026222
2021-01-07,82800,84200,82700,82900,32644642,0.008516,0.851582
2021-01-08,83300,90000,83000,88800,59013307,0.071170,7.117008


In [11]:
# 수익률 나타내는 ret 칼럼 생김
# 2021-01-04의 ret는 결측치일 수 밖에 없음, 앞에 데이터가 없으니까

In [12]:
df2=df1['ret']
df2

Date
2021-01-04         NaN
2021-01-05    1.084337
2021-01-06   -2.026222
2021-01-07    0.851582
2021-01-08    7.117008
                ...   
2021-12-24    0.750939
2021-12-27   -0.372671
2021-12-28    0.124688
2021-12-29   -1.867995
2021-12-30   -0.634518
Name: ret, Length: 248, dtype: float64

In [13]:
# dropna(): 결측치 없애겠다

In [14]:
r=df2.dropna()
r

Date
2021-01-05    1.084337
2021-01-06   -2.026222
2021-01-07    0.851582
2021-01-08    7.117008
2021-01-11    2.477477
                ...   
2021-12-24    0.750939
2021-12-27   -0.372671
2021-12-28    0.124688
2021-12-29   -1.867995
2021-12-30   -0.634518
Name: ret, Length: 247, dtype: float64

In [15]:
print(r)

Date
2021-01-05    1.084337
2021-01-06   -2.026222
2021-01-07    0.851582
2021-01-08    7.117008
2021-01-11    2.477477
                ...   
2021-12-24    0.750939
2021-12-27   -0.372671
2021-12-28    0.124688
2021-12-29   -1.867995
2021-12-30   -0.634518
Name: ret, Length: 247, dtype: float64


In [16]:
# r: 247일 동안 주가가 오르고 내린게 기록되어 있는 것
r.describe()

count    247.000000
mean      -0.014333
std        1.369006
min       -3.496503
25%       -0.731262
50%       -0.123305
75%        0.724664
max        7.117008
Name: ret, dtype: float64

In [17]:
# 2021년
# 평균 수익률 -0.01% -> 거의 0
# 근데 알고 싶은 것: 247일 동안 몇일 동안 야으이 수익률 보이는 가
# + 값 몇 개인지 세기 : (r>0).sum()
positive_count=(r>0).sum()
positive_count

110

In [18]:
# 2021년 247일 중 양의 수익률 보인 날은 110일

In [20]:
# 즉 1년동안 데이터로 추정한 '주가가 오를 확률' = p
110/247

0.44534412955465585

In [21]:
# Q: 5일간 삼전 보유하려 함, 그 5일 중에서 3일 주가가 상승할 확률 얼마인가?
# 조합 계산 math.comb(n,r)
prob=math.comb(5,3)*(0.4453**3)*(1-0.4453)**2
prob

0.2716904775542357

In [22]:
# A: 5일 삼전 보유 동안 3일은 오를 확률: 약 27%

### 3 합의 표준오차
- 이항분포는 정규분포로 근사가 가능하다
- 정규분포를 이용해서 확률을 구할 수 있다
- 그러려면 정규분포의 평균, 표준오차를 알아야함
- 강의, 필기추가

In [23]:
# 먼저 '합의 표준오차'를 '손 계산'

In [24]:
# 1 단계: 상자의 분산인 varX 계산
varX=0.4453-(0.4453**2)
varX

0.24700791

In [25]:
# 2 단계: 상자의 표준편차 sdX 계산
sdX=np.sqrt(varX)
sdX

0.4969989034193134

In [26]:
# 3 단계: 합의 표준오차 계산
# 합의 표준오차 = 상자의 표준편차*sqrt(시행횟수)
se=sdX*np.sqrt(100)
se

4.969989034193134

In [27]:
# 합의 정규분포: 평균 44.53 ( 아까의 p 네? 왜이런고징?) , 표준오차 4.969 인 분포가 된다
# 해석
# 평균적으로 44일 정도 ㅗㅇ르는데, +4.9일 정도 오차가 있다.

### 4 비율의 표준오차
> Q: 100일 동안 삼전 보유할건데, 그 100일 중 주가가 오르는 날의 비율은 몇 퍼센트?
> - cf) 247일 보유하는 동안, 이 주가가 오를 확률은 44.53% ( 위에 계산한 p )
> - 비율의 표준오차: 비율에 대한 평균과 표준오차 계산하는 것
- 필참, 강참

In [28]:
# 비율의 표준오차를 계산해보자

In [29]:
# 삼전을 247일(1년)동안 보유한다고 가정한다
# 주가가 오른 날의 비율: 평균=0.4453, 표준오차는 ? 
# 비율의 평균은 0.4453이네..?

In [30]:
# 비율의 표준오차
se_ratio=sdX/np.sqrt(247)
print(se_ratio)

0.03162328294613793


In [31]:
# 해석
# 평균적으로 오르는 비율은 44.53%, + - 3.1%정도는 왔다갔다 한다 

### 5 최대 예상손실액 구하기
> 최대 예상손실액 = VaR = Value at Risk
> - 위험관리를 위해 만들어진 개념

> VaR
> - 정상적인 시장 하에 일정기간 동안 발생할 수 있는 최대 손실금액

> 95% VaR=1,000만원
> - 해당 자산의 가치변동으로 인해 6개월동안 발생할 수 있는 최대 손실금액이 1000만원이 넘지 않을 확률이 95%
> - = 손실금액이 1000만원을 초과할 확률이 5%

> Q: 5%의 확률로 손실하는 1000만원 <- 이 값은 어케나온걸까?
> A: 과거 데이터로...
> - 삼전 과거 10년치를 봐서, 극단적인 상황이 나올 가능성이 가령 5%의 가능성으로 극단적인 - 나오는게 어느정도 인지 보는 것
> - 왼쪽 꼬리가 5% 정도인, - 수익률이 얼마인지 구하는 것

> 구하는 방법: boostrapping
- 다음 강의에...
